In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
path = r"C:\Users\bhush\OneDrive\Desktop\DSMP\Formula 1\Overall"

circuits = pd.read_csv(os.path.join(path, "circuits.csv"))
constructor_results = pd.read_csv(os.path.join(path, "constructor_results.csv"))
constructor_standings = pd.read_csv(os.path.join(path, "constructor_standings.csv"))
constructors = pd.read_csv(os.path.join(path, "constructors.csv"))
driver_standings = pd.read_csv(os.path.join(path, "driver_standings.csv"))
drivers = pd.read_csv(os.path.join(path, "drivers.csv"))
lap_times = pd.read_csv(os.path.join(path, "lap_times.csv"))
pit_stops = pd.read_csv(os.path.join(path, "pit_stops.csv"))
qualifying = pd.read_csv(os.path.join(path, "qualifying.csv"))
races = pd.read_csv(os.path.join(path, "races.csv"))
results = pd.read_csv(os.path.join(path, "results.csv"))
seasons = pd.read_csv(os.path.join(path, "seasons.csv"))
sprint_results = pd.read_csv(os.path.join(path, "sprint_results.csv"))
status = pd.read_csv(os.path.join(path, "status.csv"))

In [ ]:
drivers = drivers.replace(r"\N", np.nan, regex=False)
races = races.replace(r"\N", np.nan, regex=False)
results = results.replace(r"\N", np.nan, regex=False)

In [ ]:
# driver table
drivers["number"] = pd.to_numeric(drivers["number"])
drivers["dob"] = pd.to_datetime(drivers["dob"], errors='coerce')


# races table
races_date_cols = ["date", "fp1_date", "fp2_date", "fp3_date", "quali_date", "sprint_date"]
races_time_cols = ["time", "fp1_time", "fp2_time", "fp3_time", "quali_time", "sprint_time"]

# Convert date columns
for col in races_date_cols:
    races[col] = pd.to_datetime(races[col], errors="coerce")

# Combine date + time into full datetime columns
for d_col, t_col in zip(races_date_cols, races_time_cols):
    new_col = d_col.replace("_date", "") + "_datetime"
    races[new_col] = pd.to_datetime(
        races[d_col].astype(str) + " " + races[t_col].astype(str),
        errors="coerce"
    )

# Drop original columns
races.drop(columns=races_time_cols, inplace=True)
races.drop(columns=races_date_cols, inplace=True)

# Rename main race datetime column
races.rename(columns={"date_datetime": "race_datetime"}, inplace=True)


# Results table
# Numeric with missing values → nullable integers
int_cols = ["number", "position", "milliseconds", "fastestLap", "rank"]
for col in int_cols:
    results[col] = pd.to_numeric(results[col], errors="coerce").astype("Int64")

# Float columns
results["fastestLapSpeed"] = pd.to_numeric(results["fastestLapSpeed"], errors="coerce")

# Convert fastest lap time to timedelta
def parse_lap_time(t):
    if pd.isna(t):
        return pd.NaT
    try:
        m, s = t.split(":")
        total_seconds = int(m) * 60 + float(s)
        return pd.to_timedelta(total_seconds, unit="s")
    except:
        return pd.NaT

results["fastestLapTime"] = results["fastestLapTime"].apply(parse_lap_time)

# Optional: convert positionText to category
results["positionText"] = results["positionText"].astype("category")

In [ ]:
drivers["Driver Name"] = drivers["forename"] + " " + drivers["surname"]
drivers

### Only Driver table

In [ ]:
top_n = 20

counts = drivers["nationality"].value_counts().head(top_n)

plt.figure(figsize=(10, 6))
sns.barplot(
    x=counts.values,
    y=counts.index,
    palette="bright"
)

plt.title(f"Top {top_n} Nationalities by Number of F1 Drivers")
plt.xlabel("Number of Drivers")
plt.ylabel("Nationality")

# Add value labels
for i, v in enumerate(counts.values):
    plt.text(v + 0.3, i, str(v), va="center")

plt.tight_layout()
plt.show()

In [ ]:
drivers["number"].value_counts()

In [ ]:
drivers[drivers["number"] == 6]

In [ ]:
drivers[~drivers["number"].isna()]

### Driver + results

In [ ]:
results

In [ ]:
driver_results = results.merge(drivers, how="left", left_on="driverId", right_on="driverId")[['resultId', 'raceId', 'driverId', 'constructorId', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'statusId', 'code', 'dob', 'nationality', 'Driver Name']]
driver_results

In [ ]:
driver_results.info()

In [ ]:
driver_results.info()

In [ ]:
# Total driver points
driver_results.groupby("Driver Name")["points"].sum().sort_values(ascending=False)

In [ ]:
# Total driver races
driver_results["Driver Name"].value_counts()

In [ ]:
# points per race
(driver_results.groupby("Driver Name")["points"].sum() /
 driver_results["Driver Name"].value_counts()).sort_values(ascending=False)

In [ ]:
# Most winners
driver_results[driver_results["positionOrder"] == 1]["Driver Name"].value_counts()

In [ ]:
# Most 2nd place finish
driver_results[driver_results["positionOrder"] == 2]["Driver Name"].value_counts()

In [ ]:
# Most 3rd place finish
driver_results[driver_results["positionOrder"] == 3]["Driver Name"].value_counts()

In [ ]:
# Most podium finish
driver_results[(driver_results["positionOrder"] == 1) | (driver_results["positionOrder"] == 2) | (driver_results["positionOrder"] == 3)]["Driver Name"].value_counts()

In [93]:
# Start from results (FACT TABLE)
core_df = results.copy()

# 👤 Add Driver Info
core_df = core_df.merge(
    drivers[["driverId", "Driver Name", "code", "nationality", "dob"]],
    on="driverId",
    how="left"
)

# 📅 Add Race Context (time + circuitId)
core_df = core_df.merge(
    races[["raceId", "year", "round", "name", "race_datetime", "circuitId"]],
    on="raceId",
    how="left"
)

# 🏟️ Add Circuit / Location Info
core_df = core_df.merge(
    circuits[["circuitId", "name", "country"]],
    on="circuitId",
    how="left",
    suffixes=("", "_circuit")
)

# 🏎️ Add Constructor (Team) Info
core_df = core_df.merge(
    constructors[["constructorId", "name", "nationality"]],
    on="constructorId",
    how="left",
    suffixes=("", "_constructor")
)

# 🚦 Add Status Description (finish reason)
core_df = core_df.merge(
    status[["statusId", "status"]],
    on="statusId",
    how="left"
)


core_df = core_df.rename(columns={
    "name": "raceName",
    "name_circuit": "circuitName",
    "name_constructor": "constructorName",
    "nationality": "driverNationality",
    "nationality_constructor": "constructorNationality",
    "race_datetime": "raceDate",
    "status": "statusDescription",
    "positionOrder": "finishPosition",
    "grid": "gridPosition",
    "laps": "lapsCompleted",
    "milliseconds": "raceTime_ms"
})


core_df = core_df[[
    "raceId", "driverId", "constructorId",
    "Driver Name", "driverNationality", "dob",
    "constructorName",
    "year", "round", "raceName", "raceDate",
    "circuitName", "country",
    "gridPosition", "finishPosition", "points",
    "lapsCompleted", "raceTime_ms",
    "fastestLap", "fastestLapTime", "fastestLapSpeed",
    "statusDescription"
]]

core_df

,raceId,driverId,constructorId,Driver Name,driverNationality,dob,constructorName,year,round,raceName,...,country,gridPosition,finishPosition,points,lapsCompleted,raceTime_ms,fastestLap,fastestLapTime,fastestLapSpeed,statusDescription
0,18,1,1,Lewis Hamilton,British,1985-01-07,McLaren,2008,1,Australian Grand Prix,...,Australia,1,1,10.0,58,5690616,39,0 days 00:01:27.500000,218.300,Finished
1,18,2,2,Nick Heidfeld,German,1977-05-10,BMW Sauber,2008,1,Australian Grand Prix,...,Australia,5,2,8.0,58,5696094,41,0 days 00:01:27.700000,217.586,Finished
2,18,3,3,Nico Rosberg,German,1985-06-27,Williams,2008,1,Australian Grand Prix,...,Australia,7,3,6.0,58,5698779,41,0 days 00:01:28.100000,216.719,Finished
3,18,4,4,Fernando Alonso,Spanish,1981-07-29,Renault,2008,1,Australian Grand Prix,...,Australia,11,4,5.0,58,5707797,58,0 days 00:01:28.600000,215.464,Finished
4,18,5,1,Heikki Kovalainen,Finnish,1981-10-19,McLaren,2008,1,Australian Grand Prix,...,Australia,3,5,4.0,58,5708630,43,0 days 00:01:27.400000,218.385,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27233,1168,848,3,Alexander Albon,Thai,1996-03-23,Williams,2025,24,Abu Dhabi Grand Prix,...,UAE,17,16,0.0,58,5257796,45,0 days 00:01:28.400000,NaN,Finished
27234,1168,865,215,Isack Hadjar,French,2004-09-28,RB F1 Team,2025,24,Abu Dhabi Grand Prix,...,UAE,9,17,0.0,57,<NA>,52,0 days 00:01:29.400000,NaN,+1 Lap
27235,1168,859,215,Liam Lawson,New Zealander,2002-02-11,RB F1 Team,2025,24,Abu Dhabi Grand Prix,...,UAE,13,18,0.0,57,<NA>,39,0 days 00:01:28.900000,NaN,+1 Lap
27236,1168,842,214,Pierre Gasly,French,1996-02-07,Alpine F1 Team,2025,24,Abu Dhabi Grand Prix,...,UAE,19,19,0.0,57,<NA>,46,0 days 00:01:27.800000,NaN,+1 Lap


In [94]:
core_df.to_csv(os.path.join(path, "f1_driver_race_master.csv"), index=False)

In [95]:
core_df

,raceId,driverId,constructorId,Driver Name,driverNationality,dob,constructorName,year,round,raceName,...,country,gridPosition,finishPosition,points,lapsCompleted,raceTime_ms,fastestLap,fastestLapTime,fastestLapSpeed,statusDescription
0,18,1,1,Lewis Hamilton,British,1985-01-07,McLaren,2008,1,Australian Grand Prix,...,Australia,1,1,10.0,58,5690616,39,0 days 00:01:27.500000,218.300,Finished
1,18,2,2,Nick Heidfeld,German,1977-05-10,BMW Sauber,2008,1,Australian Grand Prix,...,Australia,5,2,8.0,58,5696094,41,0 days 00:01:27.700000,217.586,Finished
2,18,3,3,Nico Rosberg,German,1985-06-27,Williams,2008,1,Australian Grand Prix,...,Australia,7,3,6.0,58,5698779,41,0 days 00:01:28.100000,216.719,Finished
3,18,4,4,Fernando Alonso,Spanish,1981-07-29,Renault,2008,1,Australian Grand Prix,...,Australia,11,4,5.0,58,5707797,58,0 days 00:01:28.600000,215.464,Finished
4,18,5,1,Heikki Kovalainen,Finnish,1981-10-19,McLaren,2008,1,Australian Grand Prix,...,Australia,3,5,4.0,58,5708630,43,0 days 00:01:27.400000,218.385,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27233,1168,848,3,Alexander Albon,Thai,1996-03-23,Williams,2025,24,Abu Dhabi Grand Prix,...,UAE,17,16,0.0,58,5257796,45,0 days 00:01:28.400000,NaN,Finished
27234,1168,865,215,Isack Hadjar,French,2004-09-28,RB F1 Team,2025,24,Abu Dhabi Grand Prix,...,UAE,9,17,0.0,57,<NA>,52,0 days 00:01:29.400000,NaN,+1 Lap
27235,1168,859,215,Liam Lawson,New Zealander,2002-02-11,RB F1 Team,2025,24,Abu Dhabi Grand Prix,...,UAE,13,18,0.0,57,<NA>,39,0 days 00:01:28.900000,NaN,+1 Lap
27236,1168,842,214,Pierre Gasly,French,1996-02-07,Alpine F1 Team,2025,24,Abu Dhabi Grand Prix,...,UAE,19,19,0.0,57,<NA>,46,0 days 00:01:27.800000,NaN,+1 Lap


In [96]:
core_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27238 entries, 0 to 27237
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   raceId             27238 non-null  int64          
 1   driverId           27238 non-null  int64          
 2   constructorId      27238 non-null  int64          
 3   Driver Name        27238 non-null  object         
 4   driverNationality  27238 non-null  object         
 5   dob                27238 non-null  datetime64[ns] 
 6   constructorName    27238 non-null  object         
 7   year               27238 non-null  int64          
 8   round              27238 non-null  int64          
 9   raceName           27238 non-null  object         
 10  raceDate           8769 non-null   datetime64[ns] 
 11  circuitName        27238 non-null  object         
 12  country            27238 non-null  object         
 13  gridPosition       27238 non-null  int64      

In [101]:
core_df[core_df["raceTime_ms"].isna()]["year"].value_counts().sort_values()

year
2009    121
2023    134
1957    136
1969    139
1950    144
       ... 
1992    410
1988    422
1990    454
1991    454
1989    555
Name: count, Length: 76, dtype: int64

In [ ]:
core_df[["raceId", "driverId"]].duplicated().sum()

np.int64(91)

In [112]:
core_df[core_df[["raceId", "driverId"]].duplicated()].sort_values(["driverId", "raceId"])

,raceId,driverId,constructorId,Driver Name,driverNationality,dob,constructorName,year,round,raceName,...,country,gridPosition,finishPosition,points,lapsCompleted,raceTime_ms,fastestLap,fastestLapTime,fastestLapSpeed,statusDescription
13191,540,229,57,Harald Ertl,Austrian,1948-08-31,Ensign,1978,14,Italian Grand Prix,...,Italy,0,29,0.0,0,<NA>,<NA>,NaT,NaN,Did not prequalify
20287,779,356,87,Jack Brabham,Australian,1926-04-02,Cooper,1957,4,French Grand Prix,...,France,15,7,0.0,68,<NA>,<NA>,NaT,NaN,+9 Laps
24298,717,373,172,Jim Clark,British,1936-03-04,Lotus-Climax,1964,9,United States Grand Prix,...,USA,1,12,0.0,54,<NA>,<NA>,NaT,NaN,Injection
24297,745,418,172,Masten Gregory,American,1932-02-29,Lotus-Climax,1961,8,United States Grand Prix,...,USA,15,11,0.0,92,<NA>,<NA>,NaT,NaN,+8 Laps
20244,792,427,6,Maurice Trintignant,French,1917-10-30,Ferrari,1955,1,Argentine Grand Prix,...,Argentina,1,2,2.0,96,10928200,<NA>,NaT,NaN,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20233,800,702,134,Bob Scott,American,1928-10-04,Schroeder,1954,2,Indianapolis 500,...,USA,23,25,0.0,165,<NA>,<NA>,NaT,NaN,Suspension
20199,809,702,113,Bob Scott,American,1928-10-04,Kurtis Kraft,1953,2,Indianapolis 500,...,USA,19,12,0.0,190,<NA>,<NA>,NaT,NaN,+10 Laps
20236,800,730,134,George Fonder,American,1917-06-22,Schroeder,1954,2,Indianapolis 500,...,USA,26,31,0.0,101,<NA>,<NA>,NaT,NaN,Brakes
20192,823,759,105,Jan Flinterman,Dutch,1919-10-02,Maserati,1952,7,Dutch Grand Prix,...,Netherlands,16,9,0.0,83,<NA>,<NA>,NaT,NaN,+7 Laps


In [118]:
core_df[(core_df["raceId"] == 800) & (core_df["driverId"] == 612)]

,raceId,driverId,constructorId,Driver Name,driverNationality,dob,constructorName,year,round,raceName,...,country,gridPosition,finishPosition,points,lapsCompleted,raceTime_ms,fastestLap,fastestLapTime,fastestLapSpeed,statusDescription
19255,800,612,134,Andy Linden,American,1922-04-05,Schroeder,1954,2,Indianapolis 500,...,USA,23,25,0.0,165,<NA>,<NA>,NaT,NaN,Suspension
20219,800,612,113,Andy Linden,American,1922-04-05,Kurtis Kraft,1954,2,Indianapolis 500,...,USA,27,11,0.0,200,14259460,<NA>,NaT,NaN,Finished
20231,800,612,138,Andy Linden,American,1922-04-05,Nichels,1954,2,Indianapolis 500,...,USA,4,24,0.0,165,<NA>,<NA>,NaT,NaN,Retired


In [117]:
core_df[core_df["points"] == 50]

,raceId,driverId,constructorId,Driver Name,driverNationality,dob,constructorName,year,round,raceName,...,country,gridPosition,finishPosition,points,lapsCompleted,raceTime_ms,fastestLap,fastestLapTime,fastestLapSpeed,statusDescription
22514,918,1,131,Lewis Hamilton,British,1985-01-07,Mercedes,2014,19,Abu Dhabi Grand Prix,...,UAE,2,1,50.0,55,5942619,49,0 days 00:01:45.600000,189.342,Finished
